In [1]:
import { ApiPromise, SubmittableResult, WsProvider } from "@polkadot/api";
import testKeyring from "@polkadot/keyring/testing";
import { randomAsU8a } from "@polkadot/util-crypto";
import { KeyringPair } from "@polkadot/keyring/types";
import { Address } from "@polkadot/types/interfaces";
import { Abi } from '@polkadot/api-contract';
import BN from "bn.js";

import { CHARLIE as CHARLIE_ADDRESS, DOT, CREATION_FEE, WSURL } from "./tests/consts";
import {
  callContract,
  instantiate,
  getContractStorage,
  putCode,
  rpcContract,
} from "./tests/utils";

// This is a test account that is going to be created and funded before each test.
const keyring = testKeyring({ type: "sr25519" });
const CHARLIE = keyring.getPair(CHARLIE_ADDRESS);
const randomSeed = randomAsU8a(32);
let contractCreator: KeyringPair;
let api: ApiPromise;

In [2]:
api = await ApiPromise.create({ provider: new WsProvider(WSURL) });
contractCreator = keyring.addFromSeed(randomSeed);

api.tx.balances
  .transfer(contractCreator.address, CREATION_FEE.muln(5))
  .signAndSend(CHARLIE, (result: SubmittableResult): void => {
    if (
      result.status.isInBlock &&
      result.findRecord("system", "ExtrinsicSuccess")
    ) {
      console.log("New test account has been created.");
      // done();
    }
  });

Unknown types found, no types for Releases


Promise { <pending> }
New test account has been created.


In [3]:
const metadata = require("./contracts/solidity/flipper/flipper.json");
const abi = new Abi(api.registry, metadata);

const STORAGE_KEY = (new Uint8Array(32)).fill(0);
// Deploy contract code on chain and retrieve the code hash
const codeHash = await putCode(
  api,
  contractCreator,
  "../contracts/solidity/flipper/flipper.wasm"
);

In [10]:
codeHash==undefined

false


In [4]:
const address: Address = await instantiate(
      api,
      contractCreator,
      codeHash,
      abi.constructors[0](1),
      CREATION_FEE
    );
address

Type [Uint8Array] [
  36,
  253,
  100,
  148,
  30,
  183,
  21,
  11,
  126,
  144,
  223,
  151,
  125,
  240,
  136,
  183,
  53,
  155,
  231,
  214,
  222,
  33,
  181,
  193,
  161,
  138,
  67,
  236,
  71,
  220,
  110,
  193,
  registry: TypeRegistry {} ]


In [5]:
const initialValue: Uint8Array = await getContractStorage(
      api,
      address,
      STORAGE_KEY
    );
initialValue

0x3a6368696c645f73746f726167653a64656661756c743afea4179dae130ba991c4421b75d600aff46237df8ee52b7ec51624482172d009
Type [Uint8Array] [ 1, registry: TypeRegistry {} ]
Type [Uint8Array] [ 1, registry: TypeRegistry {} ]


In [6]:
initialValue.toString()

0x01


In [7]:
await callContract(api, contractCreator, address, abi.messages.flip());

In [8]:
const newValue = await getContractStorage(api, address, STORAGE_KEY);
newValue.toString()

0x3a6368696c645f73746f726167653a64656661756c743afea4179dae130ba991c4421b75d600aff46237df8ee52b7ec51624482172d009
Type [Uint8Array] [ 0, registry: TypeRegistry {} ]
0x00


In [9]:
let res = await rpcContract(api, address, abi.messages.get());
res.toString()

0x00


In [10]:
await callContract(api, contractCreator, address, abi.messages.flip());
const flipBack = await getContractStorage(api, address, STORAGE_KEY);
flipBack.toString()

0x3a6368696c645f73746f726167653a64656661756c743afea4179dae130ba991c4421b75d600aff46237df8ee52b7ec51624482172d009
Type [Uint8Array] [ 1, registry: TypeRegistry {} ]
0x01


In [11]:
res = await rpcContract(api, address, abi.messages.get());
res.toString()

0x01
